In [26]:
import os, getpass
import openai
import sys
from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import json
import ollama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import ollama

In [22]:
sys.path.append('../..')
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']

In [23]:
# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("pdf/book.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [57]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150,
    length_function=len,
    is_separator_regex=False,
)
splits = text_splitter.split_documents(docs)

In [7]:
#!pip install langchain-openai

DEPRECATION: Loading egg at /Users/ivy/anaconda3/lib/python3.11/site-packages/binwalk-2.3.3-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [8]:
#embedding = OpenAIEmbeddings()
# persist_directory = 'docs/chroma/'
# !rm -rf ./docs/chroma  # remove old database files if any
# vectordb = Chroma.from_documents(
#     documents=splits,
#     embedding=embedding,
#     persist_directory=persist_directory
# )

In [28]:
import ollama
embedding = ollama.embeddings(
  model='mxbai-embed-large',# also nomic-embed-text avilable
  prompt='Llamas are members of the camelid family',
)
persist_directory = 'docs/chroma/'

In [31]:
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
client = chromadb.Client()
collection = client.create_collection(name="docs")

In [69]:
splits[5].page_content

'help in that regard. It provides a road map for getting the product-market ﬁt as tight as possible. There\nare many considerations in this process and this book captures them well and provides practical\nguidance on how to resolve them. ”\n—Tom Byers, Entrepreneurship Professorship Endowed Chair in the\nStanford School of Engineering; Faculty Director, Stanford Technology Ventures Program\n“This is an excellent practical guide for entrepreneurs so they can see the whole process and not miss\ncritical steps as they bring products to market. Growing out of the actual experience of teaching MITstudents, it adds to the growing body of thoughtful literature in the ﬁeld that bodes well for the\nconsistent development of young entrepreneurs.”\n—Joe Lassiter, Faculty Chair of the Harvard Innovation Lab,\nand Heinz Professor of Management Practice at the Harvard Business School'

In [70]:
# store each document in a vector embedding database
for i, d in enumerate(splits):
    page_content = str(d.page_content)    
    try:
        response = ollama.embeddings(
            model="mxbai-embed-large", 
            prompt=page_content
        )
        embedding = response["embedding"]
        collection.add(
            ids=[str(i)],
            embeddings=[embedding],
            documents=[page_content], 
        )
    except Exception as e:
        print(f"Error processing document {i}: {e}")

Processing document 0
Processing document 1
Processing document 2
Processing document 3
Processing document 4
Processing document 5
Processing document 6
Processing document 7
Processing document 8
Processing document 9
Processing document 10
Processing document 11
Processing document 12
Processing document 13
Processing document 14
Processing document 15
Processing document 16
Processing document 17
Processing document 18
Processing document 19
Processing document 20
Processing document 21
Processing document 22
Processing document 23
Processing document 24
Processing document 25
Processing document 26
Processing document 27
Processing document 28
Processing document 29
Processing document 30
Processing document 31
Processing document 32
Processing document 33
Processing document 34
Processing document 35
Processing document 36
Processing document 37
Processing document 38
Processing document 39
Processing document 40
Processing document 41
Processing document 42
Processing document 4

In [71]:
question = "Describe a persona about the market segment adults between 20 and 25?"
docs = vectordb.similarity_search(question,k=3)
docs[0].page_content

'online males between the ages of 18 and 34. This segment was a very attractive group to capture,\nbecause liquor, automotive, and electronics companies were anxious to advertise to this demographic\nas they started to earn signi ﬁcant incomes and form buying habits that might last a lifetime. The team\ndid its primary market research and found there was receptivity. They also used some secondaryresearch to learn more about their demographic.\nFigure 3.1 shows a key component of their End User Pro ﬁle.\nThe charts presented in the ﬁgure indicate that they were able to use certain metrics to narrow\ntheir focus to 25-to-34-year-old males making over $75K per year. Implicit in this choice is the team\nwill be deselecting the other demographics and pursuing only one demographic to start. The criteria\nof“target audience interests ”is a good example of needing to confront the brutally honest facts about\nyour demographic, rather than looking at the end user through rose-colored glasses. Fr

### LLM

In [72]:
### LLM
local_llm = 'llama3.2:1b-instruct-fp16'
llm = ChatOllama(model=local_llm, temperature=0)

### Router


In [ ]:
def test():
    llm_json_mode = ChatOllama(model=local_llm, temperature=0, format='json')
    # sil bunu
    # document_content_description = "Lecture notes"
    # metadata_field_info = [
    #     AttributeInfo(
    #         name="source",
    #         description="The lecture the chunk is from, should be one of `docs/book.pdf`",
    #         type="string",
    #     ),
    # ]
    # Prompt 
    router_instructions = """You are an expert at routing a user question to a vectorstore or web search.
    
    The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
                                        
    Use the vectorstore for questions on these topics. For all else, and especially for current events, use web-search.
    
    Return JSON with single key, datasource, that is 'websearch' or 'vectorstore' depending on the question."""
    
    # Test router
    test_web_search = llm_json_mode.invoke([SystemMessage(content=router_instructions)] + [HumanMessage(content="Who is favored to win the NFC Championship game in the 2024 season?")])
    test_web_search_2 = llm_json_mode.invoke([SystemMessage(content=router_instructions)] + [HumanMessage(content="What are the models released today for llama3.2?")])
    test_vector_store = llm_json_mode.invoke([SystemMessage(content=router_instructions)] + [HumanMessage(content="What are the types of agent memory?")])
    print(json.loads(test_web_search.content), json.loads(test_web_search_2.content), json.loads(test_vector_store.content))

In [82]:
# create chain 
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [83]:
#test 1
question = "what is beachead market? and which step is it in?"

result = qa_chain({"query": question})
result["result"]

'According to the text:\n\n* A beachhead market is one of the critical steps in the process of narrowing down a focus and attention to one specific area of attack.\n* It involves selecting a top 1-12 market opportunity from your initial list, further segmenting that market to determine the "beachhead market", which is the most promising and profitable market.\n\nThis step is Step 2.'

In [75]:
# prepate template
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [76]:
# test template
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
question = 'what is selectable market in step 1'
result = qa_chain({"query": question})
result["result"]


'thanks for asking! \n\nA selectable market refers to the first market that you can realistically and efficiently enter with your existing resources, without having to wait for a larger or more established market to mature. It\'s the initial market where you can gain high exposure among potential customers and test your product or service before expanding into other markets.\n\nIn other words, it\'s the "first shot" at success in a new market, rather than trying to enter a large or well-established market that may take time to develop.'

In [77]:
# show where it is 
result["source_documents"][0]

Document(metadata={'page': 58, 'source': 'pdf/book.pdf'}, page_content='STEP 2\nSelect a Beachhead Market\n41')

In [79]:
# test but with info. 
# Rag doesnt work here
# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
# {context}
# Question: {question}
# Helpful Answer:"""
# question = 'what is beachead market?'
# qa_chain = RetrievalQA.from_chain_type(llm,
#                                        retriever=vectordb.as_retriever(),
#                                        return_source_documents=True,
#                                        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
# 
# 
# result = qa_chain({"query": question})
# result["result"]

"I don't know, thanks for asking!"

In [80]:
### Generate this is good.

# Prompt
rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Test
docs = vectordb.as_retriever().invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

A beachhead market refers to a position or state where a business has established itself with positive cash flow before it runs out, allowing for quick achievement of positive word of mouth (WOM) that can be a source of success or failure. In military operations, a beachhead strategy involves establishing a base of operations in enemy territory to launch further attacks and capture adjacent areas. For entrepreneurs, identifying a beachhead market is crucial as it enables them to establish a strong foundation for future growth and expansion.


In [37]:
from langchain.agents import AgentType
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.callbacks import StreamlitCallbackHandler
import streamlit as st
import pandas as pd
import os
file_formats = {
    "csv": pd.read_csv,
}
st.set_page_config(page_title="LangChain: Chat with Pandas DataFrame", page_icon="🦜")
st.title("🦜 LangChain: Chat with Pandas DataFrame")

uploaded_file = st.file_uploader(
    "Upload a Data file",
    type=list(file_formats.keys()),
    help="Various File formats are Support",
    #on_change=clear_submit,
)

df = pd.read_csv('test.csv')

pandas_df_agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
)
    
with st.chat_message("assistant"):
    st_cb = StreamlitCallbackHandler(st.container(), expand_new_thoughts=False)
    response = pandas_df_agent.run(st.session_state.messages, callbacks=[st_cb])
    st.session_state.messages.append({"role": "assistant", "content": response})
    st.write(response)

ValueError: This agent relies on access to a python repl tool which can execute arbitrary code. This can be dangerous and requires a specially sandboxed environment to be safely used. Please read the security notice in the doc-string of this function. You must opt-in to use this functionality by setting allow_dangerous_code=True.For general security guidelines, please see: https://python.langchain.com/v0.2/docs/security/